In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import re

In [40]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import Sequential
from keras import Input as Kinput

In [54]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices(), tf.test.gpu_device_name()

([name: "/device:CPU:0"
  device_type: "CPU"
  memory_limit: 268435456
  locality {
  }
  incarnation: 11886033282141896260, name: "/device:XLA_CPU:0"
  device_type: "XLA_CPU"
  memory_limit: 17179869184
  locality {
  }
  incarnation: 17107182767047603321
  physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
  device_type: "XLA_GPU"
  memory_limit: 17179869184
  locality {
  }
  incarnation: 14446573696053546960
  physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
  device_type: "GPU"
  memory_limit: 11150726272
  locality {
    bus_id: 1
    links {
    }
  }
  incarnation: 4603618681831649000
  physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"],
 '/device:GPU:0')

In [41]:
df = pd.read_csv("/content/drive/My Drive/data/insincereQuora/train.csv")

In [42]:
df.shape

(1306122, 3)

In [43]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    # text = text.split()

    return text

In [44]:
df['question_text'] = df['question_text'].apply(text_to_word_list)

In [45]:
df_train, df_test = train_test_split(df, test_size=0.20, random_state=2018)

In [46]:
# df_train, df_valid = train_test_split(df_train, test_size=0.10, random_state=2018)

In [47]:
max_length = 100
embed_size = 300
max_vocab_size = 50000

In [48]:
train = df_train["question_text"].fillna("_na_").values
test = df_test["question_text"].fillna("_na_").values
# valid = df_valid["question_text"].fillna("_na_").values

In [49]:
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(list(train))

In [50]:
train_X = tokenizer.texts_to_sequences(train)
test_X = tokenizer.texts_to_sequences(test)
# valid_X = tokenizer.texts_to_sequences(valid)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=max_length)
test_X = pad_sequences(test_X, maxlen=max_length)
# valid_X = pad_sequences(valid_X, maxlen=max_length)

## Get the target values
train_y = df_train['target'].values
test_y = df_test['target'].values
# valid_y = df_valid['target'].values

In [51]:
inp = Input(shape=(max_length,))
x = Embedding(max_vocab_size, embed_size)(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140160    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17  

In [62]:
model.fit(train_X, train_y, batch_size=2048, epochs=2, validation_data=(test_X, test_y))

Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
1044897/1044897 [==============================] - 1080s 1ms/step - loss: 0.1313 - accuracy: 0.9507 - val_loss: 0.1087 - val_accuracy: 0.9557
Epoch 2/2
1044897/1044897 [==============================] - 1072s 1ms/step - loss: 0.1067 - accuracy: 0.9573 - val_loss: 0.1073 - val_accuracy: 0.9574


In [63]:
pred_prob = model.predict([test_X], batch_size=2048, verbose=1)
max_acc = 0
at_thresh = 0
for thresh in np.arange(0.1, 0.701, 0.01):
    thresh = np.round(thresh, 2)
    acc = metrics.f1_score(test_y, (pred_prob>thresh).astype(int))
    if acc > max_acc:
      max_acc = acc
      at_thresh = thresh
print(" Max F1 score is {0} found at threshold {1}".format(max_acc, at_thresh))

261225/261225 [==============================] - 47s 181us/step
 Max F1 score is 0.654288586106768 found at threshold 0.654288586106768


In [21]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-07-03 17:19:13--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  32.6MB/s    in 49s     

2020-07-03 17:20:02 (32.3 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [19]:
# !pip install gensim
from gensim.models import KeyedVectors

In [22]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [37]:
# lower down
# remove punctutations
# take ensemble of models
# try lstm, gru
# try conv layer after lstm / gru


In [64]:
word_index = tokenizer.word_index
nb_words = min(max_vocab_size, len(word_index))
embedding_matrix = np.random.normal(0, 1, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_vocab_size: continue
    if word in word2vec.vocab:  
      embedding_vector = word2vec[word]
    else:
      embedding_vector = None
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [65]:
inp = Input(shape=(max_length,))
x = Embedding(max_vocab_size, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 100, 128)          140160    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 17  

In [66]:
model.fit(train_X, train_y, batch_size=2048, epochs=2, validation_data=(test_X, test_y))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
1044897/1044897 [==============================] - 425s 407us/step - loss: 0.1367 - accuracy: 0.9470 - val_loss: 0.1082 - val_accuracy: 0.9564
Epoch 2/2
1044897/1044897 [==============================] - 419s 401us/step - loss: 0.1032 - accuracy: 0.9584 - val_loss: 0.1048 - val_accuracy: 0.9577


In [67]:
pred_prob = model.predict([test_X], batch_size=2048, verbose=1)
max_acc = 0
at_thresh = 0
for thresh in np.arange(0.1, 0.701, 0.01):
    thresh = np.round(thresh, 2)
    acc = metrics.f1_score(test_y, (pred_prob>thresh).astype(int))
    if acc > max_acc:
      max_acc = acc
      at_thresh = thresh
print(" Max F1 score is {0} found at threshold {1}".format(max_acc, at_thresh))

261225/261225 [==============================] - 20s 75us/step
 Max F1 score is 0.657105092944121 found at threshold 0.657105092944121


In [68]:
model = Sequential()
model.add(Embedding(max_vocab_size, embed_size, input_length = max_length, weights=[embedding_matrix]))
# Bidirectional(GRU(64, return_sequences=True))
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout_U = 0.2, dropout_W = 0.2)))  # returns a sequence of vectors of dimension 32
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout_U = 0.2, dropout_W = 0.2)))  # returns a sequence of vectors of dimension 32
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout_U = 0.2, dropout_W = 0.2)))  # return a single vector of dimension 32
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 100, 128)          98816     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 100, 128)          98816     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)              

In [69]:
model.fit(train_X, train_y, batch_size=2048, epochs=2, validation_data=(test_X, test_y))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1044897 samples, validate on 261225 samples
Epoch 1/2
1044897/1044897 [==============================] - 1059s 1ms/step - loss: 0.1361 - accuracy: 0.9474 - val_loss: 0.1094 - val_accuracy: 0.9558
Epoch 2/2
1044897/1044897 [==============================] - 1058s 1ms/step - loss: 0.1084 - accuracy: 0.9554 - val_loss: 0.1062 - val_accuracy: 0.9568


In [70]:
pred_prob = model.predict([test_X], batch_size=2048, verbose=1)
max_acc = 0
at_thresh = 0
for thresh in np.arange(0.1, 0.701, 0.01):
    thresh = np.round(thresh, 2)
    acc = metrics.f1_score(test_y, (pred_prob>thresh).astype(int))
    if acc > max_acc:
      max_acc = acc
      at_thresh = thresh
print(" Max F1 score is {0} found at threshold {1}".format(max_acc, at_thresh))

261225/261225 [==============================] - 48s 183us/step
 Max F1 score is 0.6519388407650718 found at threshold 0.6519388407650718
